#### Code to train models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [3]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm

In [4]:
from params import *

In [5]:
from data.preparation import prepare_dataframe, handle_duplicates
from data.dataset import CovidDetDataset, CovidClsDataset
from data.transforms import get_transfos_det, get_transfos_cls

from model_zoo.models import get_model

from utils.plot import plot_sample
from utils.boxes import treat_boxes
from utils.logger import prepare_log_folder, save_config, create_logger, update_overall_logs

from training.main import k_fold

## Data

### Load

In [6]:
df = prepare_dataframe().copy()

In [ ]:
clusts = np.load("../output/clusts.npy", allow_pickle=True)
found = np.load("../output/found.npy")
transpositions = np.load("../output/transpositions.npy", allow_pickle=True)

df = handle_duplicates(df, clusts, transpositions, plot=False)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x="label", hue="study_label", data=df)
plt.show()

plt.figure(figsize=(15, 5))
sns.countplot(x="study_label", hue="label", data=df)
plt.show()

### Dataset

In [ ]:
transforms = get_transfos_det(augment=True, visualize=True, bbox_format="yolo")

In [ ]:
dataset = CovidDetDataset(df, DATA_PATH + f"train_{SIZE}/", bbox_format="yolo", transforms=transforms)

In [ ]:
for i in np.random.choice(len(dataset), 10):
    img, mask, y, y_img, boxes = dataset[i]
    
    if isinstance(img, torch.Tensor):
        img = img.cpu().numpy().transpose(1, 2, 0)
        mask = mask.cpu().numpy()[:, :, None]

    if len(boxes):
        plt.figure(figsize=(9, 9))
        plot_sample(img, boxes, bbox_format="yolo")
#         plot_sample(mask, boxes, bbox_format="yolo")
        plt.title(
            f'{df["save_name"][i][:-4]}  -  Study target : {CLASSES[int(y)]} - '
            f'Img target : {CLASSES_IMG[int(y_img)]}'
        )

## Model

In [ ]:
model = get_model('tf_efficientnetv2_m_in21ft1k', reduce_stride=False, num_classes=4)

In [ ]:
transforms = get_transfos_cls(augment=False, visualize=True)
dataset = CovidClsDataset(df, DATA_PATH + f"train_{SIZE}/", transforms=transforms)

x, m, y, y_img = dataset[0]

plt.figure(figsize=(9, 9))
plt.subplot(1, 2, 1)
plt.imshow(x.cpu().numpy().transpose(1, 2, 0))
plt.axis(False)
plt.subplot(1, 2, 2)
plt.imshow(m.cpu().numpy()[:, :, None])
plt.axis(False)

x = x.unsqueeze(0).float()
m = m.unsqueeze(0).float()
y = y.unsqueeze(0)
y_img = y_img.unsqueeze(0)

In [ ]:
model.encoder

In [ ]:
pred = model(x)

In [ ]:
for p in pred:
    try:
        print(p.size())
    except:
        for p_ in p:
            print(' ', p_.size())

In [ ]:
from training.optim import CovidLoss
loss = CovidLoss()

In [ ]:
y, pred[0]

In [ ]:
loss = CovidLoss()
loss(pred[0], pred[1], pred[2], y, y_img, m)

## Training

In [ ]:
BATCH_SIZES = {
    "resnext50_32x4d": 16,
    'tf_efficientnetv2_m_in21ft1k': 12,
}

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    
    size = SIZE
    bbox_format = "yolo"
    root_dir = DATA_PATH + f"train_{SIZE}/"
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    save_weights = True

    # k-fold
    k = 5
    folds_col = "kfold"
    selected_folds = [0 , 1, 2, 3, 4]

    # Model
    selected_model = 'tf_efficientnetv2_m_in21ft1k'  # "resnext50_32x4d"
    num_classes = len(CLASSES)
    reduce_stride = False

    # Training
    samples_per_patient = 1
    optimizer = "Adam"
    batch_size = BATCH_SIZES[selected_model]
    epochs = 20

    lr = 1e-3
    warmup_prop = 0.05
    val_bs = batch_size * 2

    first_epoch_eval = 0
    
    mix = "cutmix"
    mix_proba = 0.  # TODO
    mix_alpha = 5

    name = "model"

In [ ]:
DEBUG = True
log_folder = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    save_config(Config, log_folder + 'config')
    df.to_csv(log_folder + 'data.csv', index=False)
    create_logger(directory=log_folder, name="logs.txt")
    
pred_oof_study, pred_oof_img = k_fold(
    Config,
    df,
    df_extra=None,
    log_folder=log_folder
)


-------------   Fold 1 / 5  -------------

    -> 5065 training images
    -> 1269 validation images
    -> 57591021 trainable parameters

Using 403 out of 422 batches by limiting to 1 samples per study.

32768.0


/nix/store/p4xh3dls97j53rscrp6gi4qsc0d6zk4a-python3-3.8.7-env/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


32768.0
32768.0
32768.0
32768.0
32768.0
32768.0
32768.0
32768.0
32768.0
32768.0
32768.0
32768.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
16384.0
